In [8]:
from glob import glob

import folium
import numpy as np
from geopandas import GeoDataFrame, points_from_xy
from numpy import sqrt
from numpy.typing import NDArray
from pandas import concat, DataFrame, DatetimeIndex, NamedAgg, read_csv, to_datetime
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances
from skmob import TrajDataFrame

In [9]:
data_files = []
for folder in glob("../1_DATA/GeoLife/Data/*/Trajectory"):
    data_files.extend(glob(f"{folder}/*.plt"))

In [10]:
columns = ('LAT','LON','NULL','ALT','DATETIME','DATE','TIME')
raw_data = concat(
    objs=[read_csv(f, skiprows=6, names=columns).assign(TRACK_ID=f) for f in data_files[:100]],
)

In [11]:
def format_datetime(df_: DataFrame) -> DataFrame:
    DATETIME = to_datetime(df_.DATE + "T" + df_.TIME)
    return (
        df_
            .assign(
                DATETIME=DATETIME,
                EPOCH=DatetimeIndex(DATETIME).asi8 // 10**9,
                TRACK_ID=df_.TRACK_ID.str.replace(pat=f"([^0-9])", repl="", regex=True)
            )
    )

def convert_to_gpd(df_: DataFrame) -> GeoDataFrame:
    return (
        GeoDataFrame(
            data=df_,
            crs="EPSG:4326",
            geometry=points_from_xy(df_.LON, df_.LAT)
        ) # type: ignore
        .rename(columns=str.upper)
        .set_geometry("GEOMETRY")
    )

def format_data(df_: DataFrame) -> GeoDataFrame:
    return convert_to_gpd(
        df_
            .pipe(format_datetime)
            .drop(columns=["DATE", "TIME", "NULL"])
            .reset_index(drop=True)
    )

data_fmt: GeoDataFrame = format_data(raw_data)

In [12]:
track_groupby = data_fmt.groupby("TRACK_ID")
track_groups = list(track_groupby.groups)

track_groupby.agg(COUNT=NamedAgg("LAT", "count")).query("COUNT>1000")

,COUNT
TRACK_ID,
108120080523014153,11841
108120080524021405,2813
108120080530065051,1912
108120080530142423,1047
108120080531130634,1142
108120080602040534,1311
108120080607094559,4487
108120080619235223,1542
108120080622042835,1333


In [13]:
trajectory: DataFrame = track_groupby.get_group("108120080607094559")

In [134]:
D = (trajectory
        .assign(MOVE_ABILITY=np.nan, DENSITY=np.nan, CORE_POINT=False, CLUSTER=-1)
        .sort_values(by="EPOCH")
        .reset_index(drop=True)
        .astype({"CLUSTER": "Int16"}))

D_points = np.deg2rad(D[["LAT", "LON"]].values)
distance_matrix = pairwise_distances(D_points, metric="haversine") * 6_371_008.7714

In [136]:
nap = 11
eps_t = 100
ei_t = 0.1
ma_t = 0.4

MA_LOC = D.columns.get_loc("MOVE_ABILITY")
C_LOC = D.columns.get_loc("CLUSTER")

def move_ability(D: DataFrame, point_index: int, nap: int) -> float:
    MA = D.iat[point_index, MA_LOC]
    if MA != -1:
        return MA
    
    point_index_i = max(point_index - nap, D.index[0])
    point_index_j = min(point_index + nap + 1, D.index[-1])
    direct_distance = distance_matrix[point_index_i][point_index_j]
    curve_distance = sum(distance_matrix[i][i+1] for i in range(point_index_i, point_index_j))
    MA = direct_distance / curve_distance
    D.iat[point_index, MA_LOC] = MA
    return MA

def get_neighbours(D: DataFrame, point_index: int) -> set[int]:
    N = set([point_index])
    for neighbour_index in D.index[point_index+1:]:
        distance_m = distance_matrix[point_index][neighbour_index]
        if distance_m <= eps_t:
            N.add(neighbour_index)
        if move_ability(D, neighbour_index, nap) >= ma_t:
            break

    return N

def expand_cluster(D: DataFrame, point_index: int, N: set[int], C: int) -> int:
    max_index = point_index
    D.iat[point_index, C_LOC] = C
    current_neighbour = 0
    while current_neighbour < len(N):
        neighbour_index = sorted(N)[current_neighbour]

        if neighbour_index > max_index:
            max_index = neighbour_index

        N1 = get_neighbours(D, point_index)
        if D.iat[neighbour_index, C_LOC] == -1 or len(N1) >= 4:
            D.iat[neighbour_index, C_LOC] = C
            if len(N1) >= 4:
                N = N.union(N1)
        current_neighbour += 1

    return max_index


C = 0

# as the points of a trajectory are time ordered, the points with an index < than max_index 
# are either identified as noise or belong to the previous cluster. Only points where the 
# index > max_index are processed.
max_index = -1

for point_index in D.index:
    # ignore points where point_index <= max_index
    if point_index <= max_index:
        continue

    # ignore points where MA >= 0.4
    if move_ability(D, point_index, nap) >= ma_t:
        # mark point as noise
        #D.iat[point_index, C_LOC] = -1
        continue

    # get neighbours within eps_t around point at point_index
    N = get_neighbours(D, point_index)
    if len(N) >= 4:
        C += 1
        max_index = expand_cluster(D, point_index, N, C)

    



In [ ]:
D.index.is_monotonic_increasing
D.index.size
D.index.argmin()
D.index.argmax()

In [18]:
from sklearn.neighbors import BallTree

In [60]:
b_tree = BallTree(data=D_points, leaf_size=2, metric="haversine")

In [69]:
D_points[0, None]

array([[0.69803913, 2.03029843]])

In [70]:
R = 6_371_008.7714

6371.0087714

In [82]:
b_tree.query_radius(X=D_points[0, None], r=10/R, count_only=True)

array([16])

In [100]:
indices, distances = b_tree.query_radius(X=D_points[0, None], r=20/R, return_distance=True, sort_results=False)

In [101]:
indices

array([array([44, 43,  0, 30, 42, 37, 35, 36, 32, 31, 41, 40, 29, 34, 38, 39, 33,
              28,  2,  3,  1, 27])                                               ],
      dtype=object)

In [102]:
distances * R

array([array([10.86171577,  3.39873426,  0.        ,  7.28308335,  5.26159746,
               8.59156039,  8.59366636,  8.67393533,  7.96055477,  7.75723409,
               8.26161881,  8.53544951,  8.32846961,  8.45229731,  8.57662496,
               8.45727789,  8.19928711, 12.36609747, 16.092142  , 17.26628332,
              17.82656862, 18.57488917])                                      ],
      dtype=object)

In [99]:
distance_matrix[0][:38]

array([  0.        ,  17.82656862,  16.092142  ,  17.26628332,
       108.12676184, 110.23292441, 114.86329506, 117.97611062,
       122.27307052, 126.24645732, 126.53960036, 129.66675795,
       130.74370539, 132.97280696, 134.20775754, 137.24850397,
       137.44595433, 137.84410473, 140.25928716, 119.82875527,
       109.55628047,  97.03781981,  82.82906994,  67.2497016 ,
        51.09112667,  37.53265761,  26.91971906,  18.57488917,
        12.36609747,   8.32846961,   7.28308335,   7.75723409,
         7.96055477,   8.19928711,   8.45229731,   8.59366636,
         8.67393533,   8.59156039])

In [189]:
D.CORE_POINT.value_counts()

False    3045
True     1442
Name: CORE_POINT, dtype: int64

In [190]:
C = 0
core_point_mask = D.CORE_POINT==True
for key, group in D[core_point_mask].groupby((~core_point_mask).cumsum()):
    D.loc[group.index, "CLUSTER"] = C
    C += 1

In [137]:
D.CLUSTER.unique()

<IntegerArray>
[1, -1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
Length: 17, dtype: Int16

In [131]:
D.loc[D.CLUSTER==16]#.index.is_monotonic_increasing

,LAT,LON,ALT,DATETIME,TRACK_ID,EPOCH,GEOMETRY,MOVE_ABILITY,DENSITY,CORE_POINT,CLUSTER
2546,40.006623,116.323210,238.0,2008-06-07 13:55:27,108120080607094559,1212846927,POINT (116.32321 40.00662),NaN,NaN,False,16
2547,40.006626,116.323131,243.0,2008-06-07 13:55:29,108120080607094559,1212846929,POINT (116.32313 40.00663),NaN,NaN,False,16
2548,40.006636,116.323070,252.0,2008-06-07 13:55:31,108120080607094559,1212846931,POINT (116.32307 40.00664),NaN,NaN,False,16
2549,40.006626,116.323002,258.0,2008-06-07 13:55:33,108120080607094559,1212846933,POINT (116.32300 40.00663),NaN,NaN,False,16
2550,40.006614,116.322945,261.0,2008-06-07 13:55:35,108120080607094559,1212846935,POINT (116.32295 40.00661),NaN,NaN,False,16
...,...,...,...,...,...,...,...,...,...,...,...
4482,40.007503,116.323146,148.0,2008-06-07 15:46:13,108120080607094559,1212853573,POINT (116.32315 40.00750),NaN,NaN,False,16
4483,40.007495,116.323151,148.0,2008-06-07 15:46:15,108120080607094559,1212853575,POINT (116.32315 40.00749),NaN,NaN,False,16
4484,40.007487,116.323162,147.0,2008-06-07 15:46:17,108120080607094559,1212853577,POINT (116.32316 40.00749),NaN,NaN,False,16
4485,40.007471,116.323152,147.0,2008-06-07 15:46:19,108120080607094559,1212853579,POINT (116.32315 40.00747),NaN,NaN,False,16


In [132]:
raw_traj_df = TrajDataFrame(
    data=trajectory,
    latitude="LAT",
    longitude="LON",
    datetime="EPOCH",
    user_id="TRACK_ID",
    #trajectory_id="",
    timestamp=True
)

raw_traj_map: folium.Map = raw_traj_df.plot_trajectory(max_users=1, max_points=None)  # type: ignore
raw_traj_map.save("../RAW_TRAJECTORY.html")

In [138]:
traj_df = TrajDataFrame(
    data=D.loc[D.CLUSTER>-1],
    latitude="LAT",
    longitude="LON",
    datetime="EPOCH",
    user_id="CLUSTER",
    #trajectory_id="",
    timestamp=True
) # type: ignore

traj_map: folium.Map = traj_df.plot_trajectory(max_users=100, max_points=None) # type: ignore
traj_map.save("../TRAJECTORY.html")

In [69]:
from sklearn.preprocessing import StandardScaler

def apply_weight(values):
    scaler = StandardScaler()
    scaled_values = scaler.fit_transform(values.reshape(-1, 1))

    # Apply greater weight to values between 0 and 1
    weights = 1 / (1 + abs(scaled_values))

    return weights.flatten()

In [78]:
distances

array([ 82.12300639,  82.36369492,  82.10989154,  78.59745422,
        72.67386174,  64.19480085,  53.42372927,  40.0432124 ,
        27.09021102,  17.02411867,   9.71350023,  15.39157728,
        31.25738509,  45.62193656,  60.94431427,  77.87264998,
        95.42988067, 113.08619474, 132.41556827, 150.9079436 ,
       160.83758197])

array([0.0239371 , 0.02386715, 0.02394092, 0.02501081, 0.02704943,
       0.0306222 , 0.03679613, 0.04909162, 0.07256445, 0.11547067,
       0.20237672, 0.12771832, 0.0628903 , 0.04308862, 0.03225545,
       0.0252436 , 0.02059927, 0.01738308, 0.01484558, 0.01302639,
       0.01222218])

In [77]:
np.sum(weights * distances)

41.28151268361805

In [79]:
np.sum(distances)

1493.122513663945